# Bronze Layer – Raw Data Ingestion

The Bronze layer stores raw, unmodified source data ingested from CRM and ERP systems.
At this stage:

- No business rules are applied

- No transformations are performed

- Data is captured as-is to preserve lineage and auditability

This mirrors real-world BI architectures where Bronze acts as the system-of-record landing zone.

##Create Bronze Tables

In [0]:
%sql
USE CATALOG workspace;
USE SCHEMA bronze;

-- CRM tables

DROP TABLE IF EXISTS bronze.crm_cust_info;
CREATE TABLE workspace.bronze.crm_cust_info (
    cst_id             INT,
    cst_key            STRING,
    cst_firstname      STRING,
    cst_lastname       STRING,
    cst_marital_status STRING,
    cst_gndr           STRING,
    cst_create_date    DATE
)
USING DELTA;

DROP TABLE IF EXISTS bronze.crm_prd_info;
CREATE TABLE workspace.bronze.crm_prd_info (
    prd_id            INT,
    prd_key           STRING,
    prd_nm            STRING,
    prd_cost          INT,
    prd_line          STRING,
    prd_start_dt      DATE,
    prd_end_dt        DATE
)
USING DELTA;

DROP TABLE IF EXISTS bronze.crm_sales_details;
CREATE TABLE workspace.bronze.crm_sales_details (
    sls_ord_num       STRING,
    sls_prd_key       STRING,
    sls_cust_id       INT,
    sls_order_dt      INT,
    sls_ship_dt       INT,
    sls_due_dt        INT,
    sls_sales         INT,
    sls_quantity      INT,
    sls_price         INT
)
USING DELTA;

-- ERP tables

DROP TABLE IF EXISTS bronze.erp_cust_az12;
CREATE TABLE workspace.bronze.erp_cust_az12 (
    cid STRING,
    bdate DATE,
    gen STRING
)
USING DELTA;

DROP TABLE IF EXISTS bronze.erp_loc_a101;
CREATE TABLE workspace.bronze.erp_loc_a101 (
    cid STRING,
    cntry STRING
)
USING DELTA;

DROP TABLE IF EXISTS bronze.erp_px_cat_g1v2;
CREATE TABLE workspace.bronze.erp_px_cat_g1v2 (
    id STRING,
    cat STRING,
    subcat STRING,
    maintenance STRING
)
USING DELTA;



##Load CSV Files Into Bronze Tables

In [0]:
%sql
INSERT OVERWRITE workspace.bronze.crm_cust_info
SELECT
    cst_id,
    cst_key,
    cst_firstname,
    cst_lastname,
    cst_marital_status,
    cst_gndr,
    cst_create_date
FROM read_files(
    '/Volumes/workspace/bronze/source_systems/source_crm/cust_info.csv',
    format => 'csv',
    header => true
);

INSERT OVERWRITE workspace.bronze.crm_prd_info
SELECT
    prd_id,
    prd_key,
    prd_nm,
    prd_cost,
    prd_line,
    prd_start_dt,
    prd_end_dt
FROM read_files(
    '/Volumes/workspace/bronze/source_systems/source_crm/prd_info.csv',
    format => 'csv',
    header => true
);

INSERT OVERWRITE workspace.bronze.crm_sales_details
SELECT
    sls_ord_num,
    sls_prd_key,
    sls_cust_id,
    sls_order_dt,
    sls_ship_dt,
    sls_due_dt,
    sls_sales,
    sls_quantity,
    sls_price
FROM read_files(
    '/Volumes/workspace/bronze/source_systems/source_crm/sales_details.csv',
    format => 'csv',
    header => true
);

INSERT OVERWRITE workspace.bronze.erp_cust_az12
SELECT
    cid,
    bdate,
    gen
FROM read_files(
    '/Volumes/workspace/bronze/source_systems/source_erp/CUST_AZ12.csv',
    format => 'csv',
    header => true
);

INSERT OVERWRITE workspace.bronze.erp_loc_a101
SELECT
    cid,
    cntry
FROM read_files(
    '/Volumes/workspace/bronze/source_systems/source_erp/LOC_A101.csv',
    format => 'csv',
    header => true
);

INSERT OVERWRITE workspace.bronze.erp_px_cat_g1v2
SELECT
    id,
    cat,
    subcat,
    maintenance
FROM read_files(
    '/Volumes/workspace/bronze/source_systems/source_erp/PX_CAT_G1V2.csv',
    format => 'csv',
    header => true
);